# Import library

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy as sp
import scipy.stats as stats
import torch_optimizer as optim
import torch

import time
import os
from PIL import Image
from IPython.display import Image as Img
from IPython.display import display
from PIL import Image
import imageio
import natsort
import re
import sys
mod = sys.modules[__name__]

from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
from tsai.all import *
from tensorflow.python.client import device_lib

# Load data

In [ ]:
code_version = 'SISO_180t200'
net = 'TSAI'
net_type= 'TSAI_1Hz_imag'

In [ ]:
# Global path
global_path='D:/KHJ/HYUNDAI'
if os.path.exists(global_path):
    print('The directory already exists')
else:
    os.mkdir(global_path)
    
# Input data path
global_input_path_name=str(global_path)+'/Data/Training data'
if os.path.exists(global_input_path_name):
    print('The directory already exists')
else:
    os.mkdir(global_input_path_name)
    
# Result data path
global_result_path_name=str(global_path)+'/Result/'+str(net)+'/'+str(net_type)   
if os.path.exists(global_result_path_name):
    print('The directory already exists')
else:
    os.makedirs(global_result_path_name)

In [ ]:
# Global output file path
global_result_file_path =str(global_result_path_name)+'/'+str(net_type)+'_'+str(code_version)
if os.path.exists(global_result_file_path):
    print('The directory already exists')
else:
    os.mkdir(global_result_file_path)
    
# Scaler path
scaler_path=str(global_result_file_path)+'/scaler'  
if os.path.exists(scaler_path):
    print('The directory already exists')
else:
    os.mkdir(scaler_path)  
       
# loss path
loss_path=str(global_result_file_path)+'/loss'  
if os.path.exists(loss_path):
    print('The directory already exists')
else:
    os.mkdir(loss_path) 
    
# result path
result_path=str(global_result_file_path)+'/result'  
if os.path.exists(result_path):
    print('The directory already exists')
else:
    os.mkdir(result_path)

# Data preprocessing

In [ ]:
PG_path = str(global_input_path_name)+'/PG_repeat_Aw_interp_20_hr_p'
PG_file_list = os.listdir(PG_path)
PG_file_num = len(PG_file_list) 
PG_file_list = natsort.natsorted(PG_file_list)

PG_case_list = os.listdir(str(PG_path)+'/'+str(PG_file_list[0]))
PG_case_num = len(PG_case_list) 
PG_case_list = natsort.natsorted(PG_case_list)

print ("file_list: {}".format(PG_file_list))
print ("case_list: {}".format(PG_case_list))

## Import raw data

### XL data

In [ ]:
for i in range(PG_case_num):
    print('case : {}'.format(i+1)) 
    PG_raw_xl_path = str(PG_path)+'/'+str(PG_file_list[0])+'/'+str(PG_case_list[i])+'/XL'
    print(PG_raw_xl_path)
    PG_infile_list = os.listdir(PG_raw_xl_path)
    PG_infile_lists = natsort.natsorted(PG_infile_list)
    print(PG_infile_lists)
    k = 0
    m = 0
    for j in range(2): # 1:real, 2:imag
        if np.mod(j+1,2)==1: 
            k = k+1
            print(k)
            print('odd')
            print("Case index is :{}, and data type is imaginary".format(i+1))
            setattr(mod, 'xl_{}case_{}ch_{}_interp_raw'.format(i+1,k,2), np.loadtxt(str(PG_raw_xl_path)+'/'+str(PG_infile_lists[j]),delimiter=",").T) # 지정 객체의 속성 값 설정, setattr(object,name,value) object에 존재하는 name에 value를 결정한다.
            print(str(PG_raw_xl_path)+'/'+str(PG_infile_lists[j]))
            print("Data : {}, shape : {}".format(globals()['xl_{}case_{}ch_{}_interp_raw'.format(i+1,k,2)],
                                                           globals()['xl_{}case_{}ch_{}_interp_raw'.format(i+1,k,2)].shape))
        elif np.mod(j+1,2)==0:
            m = m+1
            print(m)
            print('even')
            print("Case index is :{}, and data type is real".format(i+1))
            setattr(mod, 'xl_{}case_{}ch_{}_interp_raw'.format(i+1,m,1), np.loadtxt(str(PG_raw_xl_path)+'/'+str(PG_infile_lists[j]),delimiter=",").T)
            print(str(PG_raw_xl_path)+'/'+str(PG_infile_lists[j]))
            print("Data : {}, shape : {}".format(globals()['xl_{}case_{}ch_{}_interp_raw'.format(i+1,m,1)],
                                                           globals()['xl_{}case_{}ch_{}_interp_raw'.format(i+1,m,1)].shape))

In [ ]:
xl_channel = 1
latent_xl_channel = 1

#### XL data real part

In [ ]:
for i in range(xl_channel): 
    xl_data = np.empty((0,xl_1case_1ch_1_interp_raw.shape[1]),int) 
    for j in range(PG_case_num):
        print("{}ch {}case".format(i+1,j+1))
        xl_datas = globals()['xl_{}case_{}ch_{}_interp_raw'.format(j+1,i+1,1)]
        xl_data = np.append(xl_data, xl_datas, axis=0)   
    setattr(mod, 'xl_allcase_{}ch_real_interp_raw'.format(i+1), np.array(xl_data))
    print("Data shape : {}".format(globals()['xl_allcase_{}ch_real_interp_raw'.format(i+1)].shape))      

In [ ]:
xl_data = np.empty((0,xl_1case_1ch_1_interp_raw.shape[1]),int)
for i in range(xl_channel): 
    print("{}ch".format(i+1))
    xl_datas = globals()['xl_allcase_{}ch_real_interp_raw'.format(i+1)]
    xl_data = np.append(xl_data, xl_datas, axis=0)   
xl_allcase_allch_real_interp_raw = np.array(xl_data)
print("Data shape : {}".format(xl_allcase_allch_real_interp_raw.shape))      

#### XL data imaginary part

In [ ]:
for i in range(xl_channel): 
    xl_data = np.empty((0,xl_1case_1ch_2_interp_raw.shape[1]),int)
    for j in range(PG_case_num):
        print("{}ch {}case".format(i+1,j+1))
        xl_datas = globals()['xl_{}case_{}ch_{}_interp_raw'.format(j+1,i+1,2)] 
        xl_data = np.append(xl_data, xl_datas, axis=0)   
    setattr(mod, 'xl_allcase_{}ch_imag_interp_raw'.format(i+1), np.array(xl_data)) 
    print("Data shape : {}".format(globals()['xl_allcase_{}ch_imag_interp_raw'.format(i+1)].shape))      

In [ ]:
xl_data = np.empty((0,xl_1case_1ch_2_interp_raw.shape[1]),int)
for i in range(xl_channel): 
    print("{}ch".format(i+1))
    xl_datas = globals()['xl_allcase_{}ch_imag_interp_raw'.format(i+1)]
    xl_data = np.append(xl_data, xl_datas, axis=0)   
xl_allcase_allch_imag_interp_raw = np.array(xl_data)
print("Data shape : {}".format(xl_allcase_allch_imag_interp_raw.shape))      

### MIC data

In [ ]:
for i in range(PG_case_num):
    print('case : {}'.format(i+1))
    PG_raw_mic_path = str(PG_path)+'/'+str(PG_file_list[0])+'/'+str(PG_case_list[i])+'/MIC'
    print(PG_raw_mic_path)
    PG_infile_list = os.listdir(PG_raw_mic_path)
    PG_infile_lists = natsort.natsorted(PG_infile_list)
    print(PG_infile_lists)
    k = 0
    m = 0
    for j in range(2): #1:real, 2:imag
        if np.mod(j+1,2)==1: 
            k = k+1
            print(k)
            print('odd')
            print("Case index is :{}, and data type is imaginary".format(i+1))
            setattr(mod, 'mic_{}case_{}ch_{}_interp_raw'.format(i+1,k,2), np.loadtxt(str(PG_raw_mic_path)+'/'+str(PG_infile_lists[j]),delimiter=",").T)
            print(str(PG_raw_mic_path)+'/'+str(PG_infile_lists[j]))
            print("Data : {}, shape : {}".format(globals()['mic_{}case_{}ch_{}_interp_raw'.format(i+1,k,2)],
                                                           globals()['mic_{}case_{}ch_{}_interp_raw'.format(i+1,k,2)].shape))
        elif np.mod(j+1,2)==0:
            m = m+1
            print(m)
            print('even')
            print("Case index is :{}, and data type is real".format(i+1))
            setattr(mod, 'mic_{}case_{}ch_{}_interp_raw'.format(i+1,m,1), np.loadtxt(str(PG_raw_mic_path)+'/'+str(PG_infile_lists[j]),delimiter=",").T)
            print(str(PG_raw_mic_path)+'/'+str(PG_infile_lists[j]))
            print("Data : {}, shape : {}".format(globals()['mic_{}case_{}ch_{}_interp_raw'.format(i+1,m,1)],
                                                           globals()['mic_{}case_{}ch_{}_interp_raw'.format(i+1,m,1)].shape))

In [ ]:
mic_channel = 1
latent_mic_channel = 1

#### MIC data real part

In [ ]:
for i in range(mic_channel): 
    mic_data = np.empty((0,mic_1case_1ch_1_interp_raw.shape[1]),int)
    for j in range(PG_case_num): 
        print("{}ch {}case".format(i+1,j+1))
        mic_datas = globals()['mic_{}case_{}ch_{}_interp_raw'.format(j+1,i+1,1)]
        mic_data = np.append(mic_data, mic_datas, axis=0)
    setattr(mod, 'mic_allcase_{}ch_real_interp_raw'.format(i+1), np.array(mic_data))
    print("Data shape : {}".format(globals()['mic_allcase_{}ch_real_interp_raw'.format(i+1)].shape))

In [ ]:
mic_data = np.empty((0,mic_1case_1ch_1_interp_raw.shape[1]),int)
for i in range(mic_channel): 
    print("{}ch".format(i+1))
    mic_datas = globals()['mic_allcase_{}ch_real_interp_raw'.format(i+1)]
    mic_data = np.append(mic_data, mic_datas, axis=0)
mic_allcase_allch_real_interp_raw = np.array(mic_data)
print("Data shape : {}".format(mic_allcase_allch_real_interp_raw.shape))

#### MIC data imaginary part

In [ ]:
for i in range(mic_channel): 
    mic_data = np.empty((0,mic_1case_1ch_2_interp_raw.shape[1]),int)
    for j in range(PG_case_num): 
        print("{}ch {}case".format(i+1,j+1))
        mic_datas = globals()['mic_{}case_{}ch_{}_interp_raw'.format(j+1,i+1,2)]
        mic_data = np.append(mic_data, mic_datas, axis=0)   
    setattr(mod, 'mic_allcase_{}ch_imag_interp_raw'.format(i+1), np.array(mic_data))
    print("Data shape : {}".format(globals()['mic_allcase_{}ch_imag_interp_raw'.format(i+1)].shape))

In [ ]:
mic_data = np.empty((0,mic_1case_1ch_2_interp_raw.shape[1]),int)
for i in range(mic_channel): 
    print("{}ch".format(i+1))
    mic_datas = globals()['mic_allcase_{}ch_imag_interp_raw'.format(i+1)]
    mic_data = np.append(mic_data, mic_datas, axis=0)   
mic_allcase_allch_imag_interp_raw = np.array(mic_data)
print("Data shape : {}".format(mic_allcase_allch_imag_interp_raw.shape))

## Define scaler

### Scaler : XL real

In [ ]:
xl_allcase_allch_real_interp_scaler = MinMaxScaler(feature_range=(0,1)) 
xl_allcase_allch_real_interp_scaler.fit(xl_allcase_allch_real_interp_raw)
std_data = np.array([xl_allcase_allch_real_interp_scaler.data_min_,xl_allcase_allch_real_interp_scaler.data_max_]) 
print("xl real MinMaxScaler : {}, shape : {}".format(std_data,std_data.shape))
np.savetxt(str(scaler_path)+'/MinMaxScaler_xl_allch_real_interp.dat',std_data,delimiter=",")

### Scaler : XL imag

In [ ]:
xl_allcase_allch_imag_interp_scaler = MinMaxScaler(feature_range=(0,1))
xl_allcase_allch_imag_interp_scaler.fit(xl_allcase_allch_imag_interp_raw)
std_data = np.array([xl_allcase_allch_imag_interp_scaler.data_min_,xl_allcase_allch_imag_interp_scaler.data_max_])
print("xl imag MinMaxScaler : {}, shape : {}".format(std_data,std_data.shape))
np.savetxt(str(scaler_path)+'/MinMaxScaler_xl_allch_imag_interp.dat',std_data,delimiter=",")

### Scaler : MIC real

In [ ]:
mic_allcase_allch_real_interp_scaler = MinMaxScaler(feature_range=(0,1))
mic_allcase_allch_real_interp_scaler.fit(mic_allcase_allch_real_interp_raw)
std_data = np.array([mic_allcase_allch_real_interp_scaler.data_min_,mic_allcase_allch_real_interp_scaler.data_max_])
print("mic real MinMaxScaler : {}, shape : {}".format(std_data,std_data.shape))
np.savetxt(str(scaler_path)+'/MinMaxScaler_mic_allch_real_interp.dat',std_data,delimiter=",")

### Scaler : MIC imag

In [ ]:
mic_allcase_allch_imag_interp_scaler = MinMaxScaler(feature_range=(0,1))
mic_allcase_allch_imag_interp_scaler.fit(mic_allcase_allch_imag_interp_raw) 
std_data = np.array([mic_allcase_allch_imag_interp_scaler.data_min_,mic_allcase_allch_imag_interp_scaler.data_max_])
print("mic imag MinMaxScaler : {}, shape : {}".format(std_data,std_data.shape))
np.savetxt(str(scaler_path)+'/MinMaxScaler_mic_allch_imag_interp.dat',std_data,delimiter=",")

## Data Scaling

### Scaling (XL)

In [ ]:
for i in range(PG_case_num):
    print('case : {}'.format(i+1))
    for j in range(xl_channel):
        for k in range(2):
            if k+1==1:
                setattr(mod,'xl_{}case_{}ch_{}_interp'.format(i+1,j+1,1), xl_allcase_allch_real_interp_scaler.transform(globals()['xl_{}case_{}ch_{}_interp_raw'.format(i+1,j+1,1)])) 
            elif k+1==2:
                setattr(mod,'xl_{}case_{}ch_{}_interp'.format(i+1,j+1,2), xl_allcase_allch_imag_interp_scaler.transform(globals()['xl_{}case_{}ch_{}_interp_raw'.format(i+1,j+1,2)]))  

### Scaling (MIC)

In [ ]:
for i in range(PG_case_num): 
    print('case : {}'.format(i+1))
    for j in range(mic_channel): 
        for k in range(2): 
            if k+1==1:
                setattr(mod,'mic_{}case_{}ch_{}_interp'.format(i+1,j+1,1), mic_allcase_allch_real_interp_scaler.transform(globals()['mic_{}case_{}ch_{}_interp_raw'.format(i+1,j+1,1)]))
            elif k+1==2:
                setattr(mod,'mic_{}case_{}ch_{}_interp'.format(i+1,j+1,2), mic_allcase_allch_imag_interp_scaler.transform(globals()['mic_{}case_{}ch_{}_interp_raw'.format(i+1,j+1,2)]))

## Data split

In [ ]:
#TSAI
window=10
future_window=10
strides = 1
freq_input = 21#97
freq_output = 21#97 

In [ ]:
training_data_index = np.array([1,2,3,4,5,6,8,9,10,11])
testing_data_index = np.array([7])

### XL data

In [ ]:
for i in range(xl_channel):
    xl_x_train,xl_y_train = np.empty((0,freq_input,window), int),np.empty((0,freq_input,future_window), int)

    for j in training_data_index: 
        data = globals()['xl_{}case_{}ch_{}_interp'.format(j,i+1,2)][:,160:181]
        xl_x_datas, xl_y_datas = SlidingWindow(window, stride=strides, horizon=future_window)(data)
        check_data(xl_x_datas,xl_y_datas)
        xl_x_train, xl_y_train = np.append(xl_x_train,xl_x_datas,axis=0), np.append(xl_y_train,xl_y_datas,axis=0)
    
    setattr(mod,'xl_x_{}ch_imag_train_interp'.format(i+1), np.array(xl_x_train))
    setattr(mod,'xl_y_{}ch_imag_train_interp'.format(i+1), np.array(xl_y_train))
    
print('(interp)Training data input: {}, output: {}'.format(xl_x_1ch_imag_train_interp.shape,xl_y_1ch_imag_train_interp.shape))

In [ ]:
for i in range(xl_channel):
    for j in training_data_index: 
        xl_x_train,xl_y_train = np.empty((0,freq_input,window), int),np.empty((0,freq_input,future_window), int)
        data = globals()['xl_{}case_{}ch_{}_interp'.format(j,i+1,2)][:,160:181]
        xl_x_datas, xl_y_datas = SlidingWindow(window, stride=window, horizon=future_window)(data)
        check_data(xl_x_datas,xl_y_datas)
        xl_x_train, xl_y_train = np.append(xl_x_train,xl_x_datas,axis=0), np.append(xl_y_train,xl_y_datas,axis=0)
      
        setattr(mod,'xl_x_{}case_{}ch_imag_train_interp'.format(j,i+1), np.array(xl_x_train))
        setattr(mod,'xl_y_{}case_{}ch_imag_train_interp'.format(j,i+1), np.array(xl_y_train))
        print(globals()['xl_x_{}case_{}ch_imag_train_interp'.format(j,i+1)].shape)
        print(globals()['xl_y_{}case_{}ch_imag_train_interp'.format(j,i+1)].shape)

In [ ]:
for i in range(xl_channel):
    xl_x_test,xl_y_test = np.empty((0,freq_input,window), int),np.empty((0,freq_input,future_window), int)
    
    for j in testing_data_index:
        data = globals()['xl_{}case_{}ch_{}_interp'.format(j,i+1,2)][:,160:181] 
        xl_x_datas, xl_y_datas = SlidingWindow(window, stride=window, horizon=future_window)(data)
        check_data(xl_x_datas,xl_y_datas)
        xl_x_test, xl_y_test = np.append(xl_x_test,xl_x_datas,axis=0), np.append(xl_y_test,xl_y_datas,axis=0)
        
    setattr(mod,'xl_x_{}ch_imag_test_interp'.format(i+1), np.array(xl_x_test))
    setattr(mod,'xl_y_{}ch_imag_test_interp'.format(i+1), np.array(xl_y_test))
    
print('(interp)testation data input: {}, output: {}'.format(xl_x_1ch_imag_test_interp.shape,xl_y_1ch_imag_test_interp.shape))

In [ ]:
for i in range(xl_channel):
    for j in testing_data_index: 
        xl_x_test,xl_y_test = np.empty((0,freq_input,window), int),np.empty((0,freq_input,future_window), int)
        data = globals()['xl_{}case_{}ch_{}_interp'.format(j,i+1,2)][:,160:181]
        xl_x_datas, xl_y_datas = SlidingWindow(window, stride=window, horizon=future_window)(data)
        check_data(xl_x_datas,xl_y_datas)
        xl_x_test, xl_y_test = np.append(xl_x_test,xl_x_datas,axis=0), np.append(xl_y_test,xl_y_datas,axis=0)
      
        setattr(mod,'xl_x_{}case_{}ch_imag_test_interp'.format(j,i+1), np.array(xl_x_test))
        setattr(mod,'xl_y_{}case_{}ch_imag_test_interp'.format(j,i+1), np.array(xl_y_test))
        print(globals()['xl_x_{}case_{}ch_imag_test_interp'.format(j,i+1)].shape)
        print(globals()['xl_y_{}case_{}ch_imag_test_interp'.format(j,i+1)].shape)

### MIC data

In [ ]:
for i in range(mic_channel):
    mic_x_train,mic_y_train = np.empty((0,freq_input,window), int),np.empty((0,freq_input,future_window), int)
    
    for j in training_data_index: 
        data = globals()['mic_{}case_{}ch_{}_interp'.format(j,i+1,2)][:,160:181]
        mic_x_datas, mic_y_datas = SlidingWindow(window, stride=strides, horizon=future_window)(data)
        check_data(mic_x_datas,mic_y_datas)
        mic_x_train, mic_y_train = np.append(mic_x_train,mic_x_datas,axis=0), np.append(mic_y_train,mic_y_datas,axis=0)
      
    setattr(mod,'mic_x_{}ch_imag_train_interp'.format(i+1), np.array(mic_x_train))
    setattr(mod,'mic_y_{}ch_imag_train_interp'.format(i+1), np.array(mic_y_train))
    
print('(interp)Training data input: {}, output: {}'.format(mic_x_1ch_imag_train_interp.shape,mic_y_1ch_imag_train_interp.shape))

In [ ]:
for i in range(mic_channel):
    for j in training_data_index: 
        mic_x_train,mic_y_train = np.empty((0,freq_input,window), int),np.empty((0,freq_input,future_window), int)
        data = globals()['mic_{}case_{}ch_{}_interp'.format(j,i+1,2)][:,160:181]
        mic_x_datas, mic_y_datas = SlidingWindow(window, stride=window, horizon=future_window)(data)
        check_data(mic_x_datas,mic_y_datas)
        mic_x_train, mic_y_train = np.append(mic_x_train,mic_x_datas,axis=0), np.append(mic_y_train,mic_y_datas,axis=0)
      
        setattr(mod,'mic_x_{}case_{}ch_imag_train_interp'.format(j,i+1), np.array(mic_x_train))
        setattr(mod,'mic_y_{}case_{}ch_imag_train_interp'.format(j,i+1), np.array(mic_y_train))
        print(globals()['mic_x_{}case_{}ch_imag_train_interp'.format(j,i+1)].shape)
        print(globals()['mic_y_{}case_{}ch_imag_train_interp'.format(j,i+1)].shape)

In [ ]:
for i in range(mic_channel):
    mic_x_test,mic_y_test = np.empty((0,freq_input,window), int),np.empty((0,freq_input,future_window), int)
    
    for j in testing_data_index:
        data = globals()['mic_{}case_{}ch_{}_interp'.format(j,i+1,2)][:,160:181]
        mic_x_datas, mic_y_datas = SlidingWindow(window, stride=window, horizon=future_window)(data)
        check_data(mic_x_datas,mic_y_datas)
        mic_x_test, mic_y_test = np.append(mic_x_test,mic_x_datas,axis=0), np.append(mic_y_test,mic_y_datas,axis=0)
        
    setattr(mod,'mic_x_{}ch_imag_test_interp'.format(i+1), np.array(mic_x_test))
    setattr(mod,'mic_y_{}ch_imag_test_interp'.format(i+1), np.array(mic_y_test))
    
print('(interp)testation data input: {}, output: {}'.format(mic_x_1ch_imag_test_interp.shape,mic_y_1ch_imag_test_interp.shape))

In [ ]:
for i in range(mic_channel):
    for j in testing_data_index: 
        mic_x_test,mic_y_test = np.empty((0,freq_input,window), int),np.empty((0,freq_input,future_window), int)
        data = globals()['mic_{}case_{}ch_{}_interp'.format(j,i+1,2)][:,160:181]
        mic_x_datas, mic_y_datas = SlidingWindow(window, stride=window, horizon=future_window)(data)
        check_data(mic_x_datas,mic_y_datas)
        mic_x_test, mic_y_test = np.append(mic_x_test,mic_x_datas,axis=0), np.append(mic_y_test,mic_y_datas,axis=0)
      
        setattr(mod,'mic_x_{}case_{}ch_imag_test_interp'.format(j,i+1), np.array(mic_x_test))
        setattr(mod,'mic_y_{}case_{}ch_imag_test_interp'.format(j,i+1), np.array(mic_y_test))
        print(globals()['mic_x_{}case_{}ch_imag_test_interp'.format(j,i+1)].shape)
        print(globals()['mic_y_{}case_{}ch_imag_test_interp'.format(j,i+1)].shape)

### Training data

In [ ]:
x_train_interp = np.concatenate((np.concatenate((xl_x_1ch_imag_train_interp,mic_x_1ch_imag_train_interp),axis=1),np.concatenate((xl_x_1ch_imag_test_interp,mic_x_1ch_imag_test_interp),axis=1)),axis=0)
y_train_interp = np.concatenate((mic_y_1ch_imag_train_interp,mic_y_1ch_imag_test_interp),axis=0)

print(x_train_interp.shape, y_train_interp.shape)
splits_interp = TimeSplitter(valid_size=389,show_plot=False)(y_train_interp) 
check_data(x_train_interp,y_train_interp,splits_interp)

# Training

In [ ]:
dls = get_ts_dls(x_train_interp, y_train_interp, splits=splits_interp, bs=2048)

## Network model

In [ ]:
model = create_model(InceptionTimePlus, dls=dls, **{'nb_filters':128, 'fc_dropout':0.2, 'depth':6, 'act':torch.nn.modules.activation.ELU})

## Define loss function and optimizer

In [ ]:
adam = wrap_optimizer(torch.optim.Adam)
print(adam)
loss = MSELossFlat()
print(loss)

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

## Build learner

In [ ]:
n_epoch=1000
learning_rate=0.0001

In [ ]:
fcst = ts_learner(dls, model, path='models',loss_func=loss, opt_func=adam, lr=learning_rate, device='cuda',metrics=[mae, accuracy_multi], cbs=[ShowGraph(), PredictionDynamics(figsize=(20, 12))])
fcst.fit_one_cycle(n_epoch)
fcst.save("tsai_1Hz_model_imag")
fcst.export("tsai_1Hz_imag.pkl")
torch.save(model, 'models/tsai_1Hz_model_arch_imag.pth')

# Evaluation 

## Training data

### Raw data check

In [ ]:
train_predss = fcst.get_X_preds(x_train_interp[splits_interp[0]])

In [ ]:
y_train_1_df = pd.DataFrame(np.concatenate((train_predss[0].numpy().reshape(-1,1),y_train_interp[splits_interp[0]].reshape(-1,1)),axis=1),columns=['Original','Predict'])

In [ ]:
xlim=(-0.1,1.1)
ylim=(-0.1,1.1)

plt.subplots(figsize=(15, 15))
plt.scatter(y_train_1_df["Original"],y_train_1_df["Predict"])
plt.xlabel('Original',fontsize=40)
plt.ylabel('Predict',fontsize=40)
plt.xlim(xlim); plt.ylim(ylim)
plt.xticks(fontsize=20); plt.yticks(fontsize=20)

plt.plot(xlim,ylim)
plt.annotate("r-squared = {:.5f}".format(r2_score(y_train_1_df["Original"], y_train_1_df["Predict"])), 
             xy=(0.1, 0.9), xycoords='axes fraction',ha='left', va='center',
             bbox={'boxstyle': 'round', 'fc': 'powderblue', 'ec': 'navy'},fontsize=40)
plt.show()

### case by predict

In [ ]:
for k in training_data_index:
    input_data = np.concatenate((globals()['xl_x_{}case_{}ch_imag_train_interp'.format(k,1)],globals()['mic_x_{}case_{}ch_imag_train_interp'.format(k,1)]),axis=1)
    output_data = fcst.get_X_preds(input_data)
    mic_only = output_data[0])
    print(mic_only.shape)
    
    mic_train_1ch_pred = np.empty((0,freq_output),int)
    for l in range(len(mic_only)):
        data = mic_only[l,:,:].T
        mic_train_1ch_pred = np.append(mic_train_1ch_pred,data,axis=0)
        print(mic_train_1ch_pred.shape)

    mic_train_1ch_pred_r = np.concatenate((np.zeros((len(mic_train_1ch_pred),160)), mic_train_1ch_pred),axis=1)
    mic_train_1ch_pred_rr = np.concatenate((mic_train_1ch_pred_r,np.zeros((len(mic_train_1ch_pred),300))),axis=1)
    mic_train_1ch_pred_rrr = mic_allcase_allch_imag_interp_scaler.inverse_transform(mic_train_1ch_pred_rr)

    predd_data = mic_train_1ch_pred_rrr[:,160:181].T # 180~200 Hz
    print(predd_data, predd_data.shape)
    
    plt.figure(figsize=(10,10))
    imag_output_data = globals()['mic_{}case_{}ch_{}_interp_raw'.format(k,1,2)][10:,160:181].T # 180~200 Hz
    plt.scatter(imag_output_data.reshape(-1,1),predd_data.reshape(-1,1))
    plt.show()
    
    for i in range(21):
        plt.figure(figsize=(40,10))
        plt.plot(imag_output_data[i,:])
        plt.plot(predd_data[i,:])
        plt.show()
    np.savetxt(str(result_path)+'/y_{}case_train_1ch_imag_data.dat'.format(k),predd_data, delimiter=",")

## Testing data

### Raw data check

In [ ]:
test_predss = fcst.get_X_preds(x_train_interp[splits_interp[1]])

In [ ]:
y_valid_1_df = pd.DataFrame(np.concatenate((test_predss.numpy().reshape(-1,1),y_train_interp[splits_interp[1]].reshape(-1,1)),axis=1),columns=['Original','Predict'])

In [ ]:
xlim=(-0.1,1.1)
ylim=(-0.1,1.1)

plt.subplots(figsize=(15, 15))
plt.scatter(y_valid_1_df["Original"],y_valid_1_df["Predict"])
plt.xlabel('Original',fontsize=40)
plt.ylabel('Predict',fontsize=40)
plt.xlim(xlim); plt.ylim(ylim)
plt.xticks(fontsize=20); plt.yticks(fontsize=20)

plt.plot(xlim,ylim)
plt.annotate("r-squared = {:.5f}".format(r2_score(y_valid_1_df["Original"], y_valid_1_df["Predict"])), 
             xy=(0.1, 0.9), xycoords='axes fraction',ha='left', va='center',
             bbox={'boxstyle': 'round', 'fc': 'powderblue', 'ec': 'navy'},fontsize=40)
plt.show()

### case by predict

In [ ]:
for k in testing_data_index:
    input_data = np.concatenate((globals()['xl_x_{}case_{}ch_imag_test_interp'.format(k,1)],globals()['mic_x_{}case_{}ch_imag_test_interp'.format(k,1)]),axis=1)
    output_data = fcst.get_X_preds(input_data)
    mic_only = output_data[0]
    print(mic_only.shape)
    
    mic_test_1ch_pred = np.empty((0,freq_output),int)
    for l in range(len(mic_only)):
        data = mic_only[l,:,:].T
        mic_test_1ch_pred = np.append(mic_test_1ch_pred,data,axis=0)
        print(mic_test_1ch_pred.shape)

    mic_test_1ch_pred_r = np.concatenate((np.zeros((len(mic_test_1ch_pred),160)), mic_test_1ch_pred),axis=1)
    mic_test_1ch_pred_rr = np.concatenate((mic_test_1ch_pred_r,np.zeros((len(mic_test_1ch_pred),300))),axis=1)
    mic_test_1ch_pred_rrr = mic_allcase_allch_imag_interp_scaler.inverse_transform(mic_test_1ch_pred_rr)

    predd_data = mic_test_1ch_pred_rrr[:,160:181].T # 180~200 Hz
    print(predd_data, predd_data.shape)
    
    plt.figure(figsize=(10,10))
    imag_output_data = globals()['mic_{}case_{}ch_{}_interp_raw'.format(k,1,2)][10:,160:181].T # 180~200 Hz
    plt.scatter(imag_output_data.reshape(-1,1),predd_data.reshape(-1,1))
    plt.show()
    
    for i in range(21):
        plt.figure(figsize=(40,10))
        plt.plot(imag_output_data[i,:])
        plt.plot(predd_data[i,:])
        plt.show()
    np.savetxt(str(result_path)+'/y_{}case_test_1ch_imag_data.dat'.format(k),predd_data, delimiter=",")